In [124]:
#Libraries:
import pandas as pd
import numpy as np
import os
import time
from pydub import AudioSegment
import librosa
import librosa.display
import matplotlib.pyplot as plt
from pydub import AudioSegment
from scipy.ndimage import zoom
from keras import Model, Sequential, layers, regularizers, optimizers, applications, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras import layers, models, Model
import tensorflow as tf
from pydub import AudioSegment
import pickle
from podcast_ad_skipper.params import *
from podcast_ad_skipper.google_cloud import *
from podcast_ad_skipper.data_preparation import *
from podcast_ad_skipper.fast_api import api_model, api_predict

Loading the files to train the model

In [34]:
def loading_model(h5_file_path):
    model = models.load_model(h5_file_path)
    return model


Evaluating the model: accuracy and loss metrics

DEMO - Testing new podcasts

In [70]:
file_path = '/Users/irenegracia/code/jenniferefox/podcast-ad-skipper/raw_data/test_files/Glucose Goddess -  The Scary New Research On Sugar!.mp3'
loaded_podcast_before_ads = AudioSegment.from_mp3(file_path)

In [71]:
def split_podcast(loaded_podcast_before_ads, i, clip_duration=5):
    start_time = i * 1000  # Convert to milliseconds
    end_time = (i + clip_duration) * 1000

    # Extract the clip from the podcast
    clip = loaded_podcast_before_ads[start_time:end_time]

    # Save the clip as a temporary wav file (for librosa to process)
    clip_file = "temp_clip.wav"
    clip.export(clip_file, format="wav")
    return clip_file

In [74]:
podcast_duration = len(loaded_podcast_before_ads) / 1000  # Duration in seconds

clip_list = []
clip_duration=5

for i in range(0, int(podcast_duration), clip_duration):
    clip_file=split_podcast(loaded_podcast_before_ads, i, clip_duration=5)
    clip_list.append(clip_file)

In [75]:
clip_to_test = clip_list[1]
clip_to_test

'temp_clip.wav'

Create spectrogram for the clip

In [76]:
spectrogram = create_spectrogram(clip_to_test)
spectrogram

array([[-78.466866, -63.154217, -50.48677 , ..., -67.37468 , -43.282986,
        -33.37443 ],
       [-74.1615  , -59.505154, -49.404324, ..., -58.96403 , -43.045418,
        -33.350037],
       [-58.772884, -47.684525, -44.299862, ..., -24.667297, -28.144846,
        -32.70377 ],
       ...,
       [-50.727753, -47.260498, -53.663475, ..., -66.9208  , -43.620926,
        -41.075504],
       [-52.733   , -52.60563 , -60.697353, ..., -70.066055, -46.492302,
        -43.802063],
       [-64.70975 , -63.893204, -71.34297 , ..., -80.      , -64.26426 ,
        -61.52074 ]], dtype=float32)

In [77]:
has_nans = np.isnan(spectrogram).any()

if has_nans:
    print("The spectrogram contains NaN values.")
else:
    print("The spectrogram does not contain NaN values.")

The spectrogram does not contain NaN values.


In [78]:
resized_spectrogram =resize_spectrogram(spectrogram, (224,224))

In [79]:
resized_spectrogram

array([[-78.466866, -69.317406, -56.54216 , ..., -53.850983, -37.82195 ,
        -33.37443 ],
       [-77.48378 , -68.33603 , -55.59845 , ..., -53.932358, -38.616093,
        -33.34935 ],
       [-72.372604, -63.72833 , -52.781033, ..., -47.963657, -37.782597,
        -33.3621  ],
       ...,
       [-51.568493, -50.64577 , -53.89457 , ..., -53.17004 , -41.947212,
        -42.00925 ],
       [-58.927116, -58.174423, -61.44693 , ..., -63.087006, -53.246033,
        -53.023945],
       [-64.70975 , -63.61013 , -66.24956 , ..., -70.506935, -61.987595,
        -61.52074 ]], dtype=float32)

In [80]:
scaled_spectrogram = minmax_scaler(resized_spectrogram)

In [81]:
scaled_spectrogram

array([[0.06684528, 0.17237023, 0.31971303, ..., 0.3507516 , 0.5356219 ,
        0.5869171 ],
       [0.07818366, 0.1836889 , 0.33059728, ..., 0.34981307, 0.5264626 ,
        0.5872064 ],
       [0.1371332 , 0.23683162, 0.36309183, ..., 0.41865286, 0.5360757 ,
        0.5870594 ],
       ...,
       [0.37707663, 0.3877188 , 0.3502489 , ..., 0.35860524, 0.48804328,
        0.48732778],
       [0.29220623, 0.30088738, 0.26314405, ..., 0.2442283 , 0.35772878,
        0.36029023],
       [0.22551247, 0.23819485, 0.20775312, ..., 0.15865083, 0.2569083 ,
        0.26229274]], dtype=float32)

In [82]:
has_nans = np.isnan(scaled_spectrogram).any()

if has_nans:
    print("The spectrogram contains NaN values.")
else:
    print("The spectrogram does not contain NaN values.")

The spectrogram does not contain NaN values.


In [83]:
reshaped_spectrogram = reshape_spectrogram(scaled_spectrogram)


In [84]:
has_nans = np.isnan(reshaped_spectrogram).any()

if has_nans:
    print("The spectrogram contains NaN values.")
else:
    print("The spectrogram does not contain NaN values.")

The spectrogram does not contain NaN values.


In [85]:
# Convert the spectrogram to a numpy array and pass it to the model
spectrogram_np = np.expand_dims(reshaped_spectrogram, axis=0)

In [133]:
spectrogram_np

array([[[[0.06684528, 0.06684528, 0.06684528],
         [0.17237023, 0.17237023, 0.17237023],
         [0.31971303, 0.31971303, 0.31971303],
         ...,
         [0.3507516 , 0.3507516 , 0.3507516 ],
         [0.5356219 , 0.5356219 , 0.5356219 ],
         [0.5869171 , 0.5869171 , 0.5869171 ]],

        [[0.07818366, 0.07818366, 0.07818366],
         [0.1836889 , 0.1836889 , 0.1836889 ],
         [0.33059728, 0.33059728, 0.33059728],
         ...,
         [0.34981307, 0.34981307, 0.34981307],
         [0.5264626 , 0.5264626 , 0.5264626 ],
         [0.5872064 , 0.5872064 , 0.5872064 ]],

        [[0.1371332 , 0.1371332 , 0.1371332 ],
         [0.23683162, 0.23683162, 0.23683162],
         [0.36309183, 0.36309183, 0.36309183],
         ...,
         [0.41865286, 0.41865286, 0.41865286],
         [0.5360757 , 0.5360757 , 0.5360757 ],
         [0.5870594 , 0.5870594 , 0.5870594 ]],

        ...,

        [[0.37707663, 0.37707663, 0.37707663],
         [0.3877188 , 0.3877188 , 0.3877188 ]

In [86]:
has_nans = np.isnan(spectrogram_np).any()

if has_nans:
    print("The spectrogram contains NaN values.")
else:
    print("The spectrogram does not contain NaN values.")

The spectrogram does not contain NaN values.


In [87]:
h5_file_path = 'model_folder/3_model_leo.h5'

In [88]:
model = loading_model(h5_file_path)

In [89]:
prediction = model.predict(spectrogram_np)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step


In [111]:
prediction

array([[0.01948208]], dtype=float32)

In [112]:
# Assume prediction = 1 to see if the other functions are working
prediction_to_test = 1

In [113]:
# Not zero exactly, could there be an issue with ad predictions being 0.9< but not quite 1? Thereby not getting added to ad_segments
# Unless prediction = 1, they won't be added to ad_segments?

In [114]:
def save_ad_segments(prediction):
    ad_segments = []
    if prediction == 1:
        ad_segments.append((i, i + clip_duration))
    return ad_segments

In [108]:
ad_segments = save_ad_segments(prediction)
ad_segments

[]

In [115]:
# Test to see if save_ad_segments works
ad_segments_test = save_ad_segments(prediction_to_test)


In [116]:
ad_segments_test

[(6025, 6030)]

In [93]:
clean_podcast = AudioSegment.empty() # Create an empty AudioSegment object
current_time = 0

In [94]:
for ad_start, ad_end in ad_segments:
    ad_start_ms = ad_start * 1000 # Convert to milliseconds
    ad_end_ms = ad_end * 1000

    print(ad_start_ms)
    print(ad_end_ms)

In [95]:
clean_podcast += loaded_podcast_before_ads[current_time:ad_start_ms] # Add the non-ad segment to the clean podcast
clean_podcast

NameError: name 'ad_start_ms' is not defined

In [96]:
predictions = []
for clip_file in clip_list:
    spectrogram = create_spectrogram(clip_file) # We already have this function
    resized_spectrogram =resize_spectrogram(spectrogram, (224,224))
    scaled_spectrogram = minmax_scaler(resized_spectrogram)
    reshaped_spectrogram = reshape_spectrogram(scaled_spectrogram)

    # Convert the spectrogram to a numpy array and pass it to the model
    spectrogram_np = np.expand_dims(reshaped_spectrogram, axis=0)  # Add batch dimension
    prediction = model.predict(spectrogram_np)
    predictions.append(prediction)

print(predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━

In [97]:
predictions

[array([[0.01948208]], dtype=float32),
 array([[0.01948208]], dtype=float32),
 array([[0.01948208]], dtype=float32),
 array([[0.01948208]], dtype=float32),
 array([[0.01948208]], dtype=float32),
 array([[0.01948208]], dtype=float32),
 array([[0.01948208]], dtype=float32),
 array([[0.01948208]], dtype=float32),
 array([[0.01948208]], dtype=float32),
 array([[0.01948208]], dtype=float32),
 array([[0.01948208]], dtype=float32),
 array([[0.01948208]], dtype=float32),
 array([[0.01948208]], dtype=float32),
 array([[0.01948208]], dtype=float32),
 array([[0.01948208]], dtype=float32),
 array([[0.01948208]], dtype=float32),
 array([[0.01948208]], dtype=float32),
 array([[0.01948208]], dtype=float32),
 array([[0.01948208]], dtype=float32),
 array([[0.01948208]], dtype=float32),
 array([[0.01948208]], dtype=float32),
 array([[0.01948208]], dtype=float32),
 array([[0.01948208]], dtype=float32),
 array([[0.01948208]], dtype=float32),
 array([[0.01948208]], dtype=float32),
 array([[0.01948208]], dt

In [98]:
series = pd.Series(predictions)

# Get value counts
value_counts = series.value_counts()

print(value_counts)

[[0.019482082]]    1206
Name: count, dtype: int64
